Preprocess data

In [11]:
import pandas as pd
from tqdm import tqdm

In [35]:
df = pd.read_csv(r'C:\Users\Clamfighter\Documents\GitHub\Product_OCR\Brandbank Extract.csv')
df=df.fillna('')
df['text'] = df[[
    'description',
    'allergyAdvice',
    'recycling',
    'recycling_other',
    'brands',
    'marketing',
    'ingredients',
    'features',
    'storage',
    'preserves']].agg(' '.join, axis=1)
df=df.drop(['name'
    ,'pl2','pl3',
    'description',
    'allergyAdvice',
    'recycling',
    'recycling_other',
    'brands',
    'marketing',
    'ingredients',
    'features',
    'storage',
    'preserves'],axis=1)
df.rename({'pl1':'category'},axis=1, inplace=True)
df.drop(df[df['category'] =='Kruidenierswaren'].index, inplace=True)#onyl 1 example of this so dropping for simplicity
df['category'] = df['category'].str.replace(" ", "_")
df = df[df['category'] != '']
df = df[df['category'] != 'Unallocated']
df.head()

,category,text
0,Dairy_&_Bread,Waitrose Fig & Raisin Loaf 400g Waitrose
1,Deli,WR Gingerbread biscuit chick each
2,Dairy_&_Bread,Waitrose Petit Parisienne 290g Waitrose
3,Fruit_&_Vegetables,Waitrose Loose Yellow Peppers Waitrose
4,Fruit_&_Vegetables,Asda Pink Lady Apples Asda


RAKE_NLTK

In [36]:
from rake_nltk import Rake

In [37]:
def get_num_words(row):
    return len(str(row['Phrase']).split())

def get_perc_phrase(row,total):
    return (float(row['Score'])/total*100)

In [38]:
def get_Rake_phrases(df):
    r = Rake()
    r.extract_keywords_from_sentences(df['text'])
    phrases=r.get_ranked_phrases_with_scores()
    df = pd.DataFrame(phrases, columns = ['Rank', 'Phrase'])
    df['Phrase'] = df['Phrase'].str.strip()
    phrase_counts = df['Phrase'].value_counts()
    phrase_counts = phrase_counts.reset_index()
    phrase_counts.columns=['Phrase','Score']
    phrase_counts['Length'] = phrase_counts.apply (lambda row: get_num_words(row), axis=1)
    phrase_counts=phrase_counts.loc[phrase_counts['Length'] <=6]
    total = float(phrase_counts['Score'].sum())
    phrase_counts['Perc'] = phrase_counts.apply (lambda row: get_perc_phrase(row,total), axis=1)
    return phrase_counts

In [39]:
df_raked = get_Rake_phrases(df)
#phrase_counts.to_csv(r'C:\Users\Clamfighter\Documents\GitHub\NLP\Impact Score Key Phrase\Extracted Phrases\Rake NLTK\Brandbank Latest Phrase Extraction.csv',index = False)

In [40]:
df_phraseCount = df_raked.loc[df_raked['Score'] != 1]

In [43]:
phrase_count=[]
phrases = df_phraseCount['Phrase']
text = df['text']
for phrase in tqdm(phrases,position=0,leave=True):
    phrase_count.append(sum(phrase in s for s in text))
phrase_count

 22%|██▏       | 50138/228459 [1:09:51<4:17:36, 11.54it/s] 

KeyboardInterrupt: 

 22%|██▏       | 50138/228459 [1:10:04<4:17:36, 11.54it/s]

In [29]:
df_phraseCount['Count'] = phrase_count
df_phraseCount.drop(['Score','Length','Perc'],axis=1, inplace=True)
df_out = df_raked.merge(df_phraseCount, on='Phrase', how='left')
df_out.fillna(1,inplace=True)
df_out

c:\Users\Clamfighter\Machine_Learning_Project\my_env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\Users\Clamfighter\Machine_Learning_Project\my_env\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Phrase,Count
0,contains,3368
1,may contain,26
2,suitable,908
3,salt,1029
4,sugar,1890
...,...,...
101393,everyone needs,1
101394,easy job,2
101395,crime threatens,2
101396,xhose,0


In [34]:
df_out.to_csv(r'C:\Users\Clamfighter\Documents\GitHub\NLP\Impact Score Key Phrase\Extracted Phrases\Rake NLTK\Brandbank Latest Phrase Extraction.csv',index = False)